This is a simple Hello World example, showing how to take a function and run it with pywren. First we import the necessary libraries. 

In [1]:
import pywren
import numpy as np
import redis
from rediscluster import StrictRedisCluster

Pywren is designed to run any existing python functions you have, in parallel, at scale, on the cloud. So first, we create an example python function. The funtion must take a single argument:

In [12]:
def my_function(x):
    return x + 7
my_function(3)

10

In [29]:
def my_function(x):
    # cluster mode disabled: connect to node endpoint 
    #client = redis.Redis(host="rediscluster.a9ith3.clustercfg.usw2.cache.amazonaws.com", port=6379)
    
    # cluster mode enabled: connect to configuration endpoint 
    startup_nodes = [{"host": "rediscluster.a9ith3.clustercfg.usw2.cache.amazonaws.com", "port": "6379"}]
    client = StrictRedisCluster(startup_nodes=startup_nodes, decode_responses=True, skip_full_coverage_check=True)

    key = "foo"+str(x)
    client.set(key, str(x))
    r = client.get(key)
    if (r is None):
        print "no such key"
        return -1
    if r == str(x):
        return "success! " + str(r)
    
    return "error " + str(r)

To start using `pywren`, we first create an executor.

In [28]:
wrenexec = pywren.default_executor()

We can call `my_function(3)` remotely via `call_async`: 

In [23]:
future = wrenexec.call_async(my_function, 3)

Future is a placeholder for the returned value from applying `my_function` to the number `3`. We can call `result` on it and get the result. Note that this will block until the remote job has completed

In [24]:
future.result()

'success!3'

You can apply `my_function` to a list of arguments, and each will be executed remotely at the same time. 

In [25]:
futures = wrenexec.map(my_function, range(10))

The pywren `get_all_results` function will wait until all of the futures are done and return their results

In [26]:
pywren.get_all_results(futures)

['success!0',
 'success!1',
 'success!2',
 'success!3',
 'success!4',
 'success!5',
 'success!6',
 'success!7',
 'success!8',
 'success!9']